In [ ]:
import os
os.chdir('/home/roberto/Documents/Titulación/Tesina')
from scripts import *

### Países disponibles:

##### -España
##### -Alemania

### Años disponibles:

##### 2008 a 2017

Correr siguiente celda para repetir toda la simulación

In [ ]:
#import multiprocessing
#from joblib import delayed, Parallel

#Parallel(n_jobs=multiprocessing.cpu_count()-1)(delayed(solar_year)(country,year) for country in paises for year in years)

Correr siguiente celda para leer archivos existentes de simulación anterior

In [ ]:
#todo = {}
#for year in years:
#    for country in paises:
#        todo[country+str(year)]=pd.read_csv('../Archivos/Output/'+country+str(year)+'.csv',parse_dates=[0])
#        todo[country+str(year)].set_index('time',inplace=True)
#        todo[country+str(year)]=todo[country+str(year)]/1000            ## Convertir a GW

Correr siguiente celda para integrar todos los datos en un único df

In [ ]:
#df = pd.concat([todo['Spain2008'],todo['Germany2008']],axis=1)

#for year in years[1:]:
#    df_aux = pd.concat([todo['Spain'+str(year)],todo['Germany'+str(year)]],axis=1)
#    df = pd.concat([df,df_aux])
    
#df.to_csv('../Archivos/Output/Combinado.csv')

Lectura del df único

In [ ]:
def leer():
    df = pd.read_csv('../Archivos/Output/Combinado.csv',parse_dates=[0])
    df = df[(df['time'] < '2018-01-01')]
    df.set_index('time',inplace=True)
    df.index.drop_duplicates(keep=False)
    return (df)

In [ ]:
df = leer()

In [ ]:
df.loc['2010-08-12']

Gráfica tres días aleatorios de producción

In [ ]:
def tres_random(df,pais):
    df['doy'] = df.index.dayofyear
    df['tiempo']=df.index.time
    df['Year'] = df.index.year
    df['ind']= df['doy'].astype(str)+"-"+ df['tiempo'].astype(str)
    df['ind']=pd.to_datetime(df['ind'],format='%j-%H:%M:%S')
    piv = pd.pivot_table(df, index=['ind'],columns=['Year'], values=[pais])
    piv = piv[(piv.index <'1901-01-01')]
    mes = np.random.choice(list(piv.index.month.unique()))
    aux = piv[piv.index.month==mes]
    split = np.array_split(aux,10)
    x = np.random.choice(range(10))
    titulo = pais+"\n"+ str(calendar.month_name[mes])
    split[x].plot(title = titulo,figsize=(20,7),xlabel = 'Date',ylabel='Solar Power [GW]', ls='--')
    plt.legend(title = "País:")
    plt.show()
    plt.close()

In [ ]:
for pais in paises:
    tres_random(df,pais)

Obtención de estadísticas descriptivas de producción total en los diez años

In [ ]:
df = leer()

In [ ]:
def descriptivas(df,pais):
    pais_1 = df[pais]
    total = {}
    for year in years:
        total[str(year)] = pais_1.loc[str(year)].sum()
    sumas = pd.DataFrame.from_dict(total,orient='index')
    titulo  = 'GWh ' + pais
    sumas.rename(columns={0:titulo},inplace=True)
    return(sumas)

In [ ]:
spa = descriptivas(df,'Spain')
spa

In [ ]:
spa.describe()

In [ ]:
ger = descriptivas(df,'Germany')
ger

In [ ]:
ger.describe()

Producción promedio, máx, min y med de un día en cada mes

In [ ]:
def Por_mes(Pais,Mes):
    df = leer()
    df['Month']=df.index.month
    df['Time']=df.index.time
    Country = df[[Pais,'Month','Time']]
    mes = Country.loc[Country['Month']== Mes]
    aux = mes.groupby('Time')
    hora =aux[Pais].agg([np.median, lambda x: np.quantile(x,0.75),lambda x: np.quantile(x,0.25)])
    hora.rename(columns={'<lambda_0>':'Perc. 0.75','<lambda_1>':'Perc. 0.25'},inplace=True)
    return(hora)

In [ ]:
def cuatrimestre(num_cuatrimestre,num_primer_mes,country):
    import calendar
    df = leer()
    max = df[country].max()
    titulo_sup =country+"\n"+str(num_primer_mes)+" - "+str(num_primer_mes+3)
    plt.figure(figsize=(60,35))
    plt.suptitle(titulo_sup,fontsize=50)

    plt.subplot(2,2,1)

    month_1 = Por_mes(country,num_primer_mes)

    x_axis = month_1.index.astype(str)

    plt.plot(x_axis,month_1['Perc. 0.75'],ls='dashdot',label="Perc. 0.75")
    plt.plot(x_axis,month_1['median'],ls='solid',label="Median")
    plt.plot(x_axis,month_1['Perc. 0.25'],ls='dashed',label="Perc. 0.25")
    titulo = str(calendar.month_name[num_primer_mes])
    plt.title(titulo,fontsize = 35)
    plt.legend(loc='upper left',fontsize='xx-large')
    plt.ylabel('Solar Power [GW]',fontsize = 35)
    plt.xlabel('Hour of day',fontsize=35)
    plt.tick_params(axis='y',labelsize=25)
    plt.minorticks_off()
    plt.ylim(bottom = 0,top = max)

    plt.subplot(2,2,2)

    month_2 = Por_mes(country,num_primer_mes+1)
    plt.plot(x_axis,month_2['Perc. 0.75'],ls='dashdot',label="Perc. 0.75")
    plt.plot(x_axis,month_2['median'],ls='solid',label="Median")
    plt.plot(x_axis,month_2['Perc. 0.25'],ls='dashed',label="Perc. 0.25")
    titulo = str(calendar.month_name[num_primer_mes+1])
    plt.title(titulo,fontsize = 35)
    plt.legend(loc='upper left',fontsize='xx-large')
    plt.ylabel('Solar Power [GW]',fontsize = 35)
    plt.tick_params(axis='y',labelsize=25)
    plt.xlabel('Hour of day',fontsize=35)
    plt.minorticks_off()
    plt.ylim(bottom = 0,top = max)

    plt.subplot(2,2,3)

    month_3 = Por_mes(country,num_primer_mes+2)
    plt.plot(x_axis,month_3['Perc. 0.75'],ls='dashdot',label="Perc. 0.75")
    plt.plot(x_axis,month_3['median'],ls='solid',label="Median")
    plt.plot(x_axis,month_3['Perc. 0.25'],ls='dashed',label="Perc. 0.25")
    titulo = str(calendar.month_name[num_primer_mes+2])
    plt.title(titulo,fontsize = 35)
    plt.legend(loc='upper left',fontsize='xx-large')
    plt.ylabel('Solar Power [GW]',fontsize = 35)
    plt.tick_params(axis='y',labelsize=25)
    plt.xlabel('Hour of day',fontsize=35)
    plt.minorticks_off()
    plt.ylim(bottom = 0,top = max)

    plt.subplot(2,2,4)

    month_4 = Por_mes(country,num_primer_mes+3)
    plt.plot(x_axis,month_4['Perc. 0.75'],ls='dashdot',label="Perc. 0.75")
    plt.plot(x_axis,month_4['median'],ls='solid',label="Median")
    plt.plot(x_axis,month_4['Perc. 0.25'],ls='dashed',label="Perc. 0.25")
    titulo = str(calendar.month_name[num_primer_mes+3])
    plt.title(titulo,fontsize = 35)
    plt.legend(loc='upper left',fontsize='xx-large')
    plt.ylabel('Solar Power [GW]',fontsize = 35)
    plt.tick_params(axis='y',labelsize=25)
    plt.xlabel('Hour of day',fontsize=35)
    plt.minorticks_off()
    plt.ylim(bottom = 0,top = max)
    
    plt.show()

Gráficas por mes

In [ ]:
cuatrimestre(1,1,'Spain')
cuatrimestre(2,5,'Spain')
cuatrimestre(3,9,'Spain')

In [ ]:
cuatrimestre(1,1,'Germany')
cuatrimestre(2,5,'Germany')
cuatrimestre(3,9,'Germany')

Mapas

In [ ]:
area_elegible('Spain',dump)

In [ ]:
area_elegible('Germany',dump)